In [1]:
#Importamos librerias
import pandas as pd
import numpy as np


In [2]:
#cargas el conjunto de datos "diabetes.csv" en una variable diabetes
diabetes = pd.read_csv("diabetes.csv")
diabetes.head(5)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

In [3]:
#obten las dimensiones del dataframe diabetes
diabetes.shape

(768, 9)

In [4]:
# Obten la cantidad de valores nulos que contiene el dataframe
diabetes.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [27]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
outcome_count = diabetes['Outcome'].value_counts()
print(outcome_count)

Outcome
0    500
1    268
Name: count, dtype: int64


In [6]:

from sklearn.utils import resample

# Separar los valores en dos grupos basados en el valor en la columna "Outcome"
df_majority = diabetes[diabetes['Outcome'] == 0]
df_minority = diabetes[diabetes['Outcome'] == 1]

# Aplicar el submuestreo al valor mayoritario (0)
df_majority_subsampled = resample(df_majority, replace=False, n_samples=outcome_count[1], random_state=42)

# Combinar los subconjuntos de datos balanceados
df_balanced = pd.concat([df_majority_subsampled, df_minority])

In [10]:
outcome_count = df_balanced['Outcome'].value_counts()
print(outcome_count)

Outcome
0    268
1    268
Name: count, dtype: int64


In [12]:
from sklearn.preprocessing import MinMaxScaler

column_scale = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age","Outcome"]
scaler = MinMaxScaler()
df_balanced[column_scale] = scaler.fit_transform(df_balanced[column_scale])

In [13]:
from sklearn.svm import SVC

X= df_balanced.iloc[:,:-1]#features
y= df_balanced.iloc[:,-1]#target
svc = SVC()

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

In [16]:
svc.fit(X_train, y_train)

SVC()

In [17]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(svc,X, y, cv=5)

print(scores)
print(scores.mean())

[0.76851852 0.71962617 0.71028037 0.70093458 0.75700935]
0.7312737971616476


In [21]:
grid = {
    'C': [0.1,1,10],
    'kernel': ['linear','rbf'],
    'gamma':[0.1,1,10]
}

In [22]:

from sklearn.model_selection import GridSearchCV
svc_cv = GridSearchCV(estimator=SVC(),param_grid=grid,cv=5)
svc_cv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf']})

In [23]:
svc_cv.best_params_

{'C': 1, 'gamma': 10, 'kernel': 'rbf'}

In [24]:
svc2 = SVC(C=1,gamma=10,kernel='rbf').fit(X_train,np.ravel(y_train))

In [25]:
import pickle
pickle_out = open("classifier.pkl","wb")
pickle.dump(svc2, pickle_out)
pickle_out.close()